In [ ]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from reentry1.ballistic import run_ballistic_simulation
from reentry1.base import  ft2m, lbfsqf2Nsqm, Pa2lbfsqf, Spacecraft
from reentry1.lifting import run_lifting_simulation
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

plt.style.use('classic')
font = {'family' : 'monospace',
        'weight' : 'regular',
        'size'   : 13}

plt.rc('font', **font) 
plt.rc('legend',fontsize=13)

%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)
# fig.canvas.layout.width = '100%'
# fig.canvas.layout.height = '900px'

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

# Ballistic Re-entry

A continuació es proveixen varies opcions per estudis de re-entrada balística. 
Sel•leccionant la "missió" custom, es poden establir tots els parametres del vehicle de reentrada i les condicions inicials. Les varies opcions referents a "beta study" es refereixen a produir gràfics simultanis per a diferents valors de beta, del coeficient balístic com es mostrava a la Figura 1 del paper d'Adams.


Tots els gràfics es produeixen a partir de les solucions de velocitat, $\gamma$, temps, range, provinents de l'integració numérica respecte de l'altitud, de les següents equacions de reentrada balística:


$$  \frac{d V}{d h}=\frac{g\left[\frac{Q}{\beta}-\sin (\gamma)\right]}{V \sin (\gamma)} \\ $$
$$ \frac{d \gamma}{d h}=\frac{\cos (\gamma)\left[-g+\frac{V^{2}}{R_{e}+h}\right]}{V^{2} \sin (\gamma)} \\ $$
$$ \frac{d t}{d h}=\frac{-1}{V \sin (\gamma)} \\ $$
$$ \frac{d r}{d h}=R_{e} \frac{d t}{d h}=\frac{d t}{d h}=\frac{-R_{e} \cos (\gamma)}{\left(R_{e}+h\right) \sin (\gamma)} \\ $$




missions:
- *beta_study*  per defecte executa i grafica els plots de la Figura 1 del paper
- *mercury* sense cao canvi de paramàmetres, per defecte executa i grafica els plots de la Figura 2 del paper.
- *custom* permet realitzar figures similars a les del paper amb total llibertat de paràmetres. Si es vol fer un estudi de coeficients balístics com al paper, s'ha d'activar l'opció *custom_betastudy*.


- opcions *parachute* es refereixen a afegir una aproximació de sistema de paracaigudes de la càpsula Orion scalat a les dimensions de la càpsula Mercury. Veure Memòria per més referencia.


Amb l'actual implementació l'Opció **orion_parachute** només pot ser seleccionada al mateix temps que **missió custom** i amb **custom_betastudy desactivat**. Nota: Opció *parachute* incrementa el temps de còmput notablement al incrementar el temps de reentrada i l'integrador haver de fer més steps.





In [ ]:
widgets.interact_manual.opts['manual_name'] = 'Run Simulation'



@widgets.interact_manual(betas= widgets.FloatRangeSlider(
    value=[100., 500.],
    min=30, max=6000, step=10,
    disabled=False,
    description='Betas:',
    readout_format='.1f',
    ), betastudy=widgets.Checkbox(
    value=None,
    description='Custom Beta study',
    disabled=False,
    indent=True
)
)
def ballistic_example( mission=["custom","beta_study","mercury", "mercury_parachute" ],custom_betastudy=False,  
                      input_units=["imperial", "metric"],plot_units=["imperial", "metric"], 
                      betas=500, W=2662.8, L_ref=6.2, A_ref=30.27, Cd=1.60, 
                      R_nose=1.0, V_0=22500.0, gamma_0=1.5, altitude=250000.0, orion_parachute=False,):


    if mission == "mercury_parachute":
        input_units = "imperial"
        W = 2662.8  + 2* 80 + 3*103 # lbf
        L_ref = 6.2 #ft
        A_ref = 30.27 # ft^2 
        Cd = 1.60
        R_nose = 1.0 # ft
        initial_conditions = [23_000, 1.5, 0.0, 0.0] # [V_ini [ft], gamma_ini[deg], time_ini, range_ini]
        altitude = 250_000.0 # ft
        spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl=None, 
                                parachute=True, imperial_units=True, beta_study=False)   # spacecraft Class stores specs and handles unit conversions
        betas = Pa2lbfsqf(spacecraft.beta)
        print("Drogue Area", ft2m(ft2m(2* (23/2.2/2)**2*pi))) # ft^2 m^2)
        print("main area" , ft2m(ft2m(3* (116/2.2/2)**2 * pi))) 

    elif mission == "mercury":
        input_units = "imperial"
        W = 2662.8  # lbf
        L_ref = 6.2 #ft
        A_ref = 30.27 # ft^2 
        Cd = 1.60
        R_nose = 1.0 # ft
        initial_conditions = [23_000, 1.5, 0.0, 0.0] # [V_ini [ft], gamma_ini[deg], time_ini, range_ini]
        altitude = 250_000.0 # ft
        spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl=None, 
                                parachute=False, imperial_units=True, beta_study=False)   # spacecraft Class stores specs and handles unit conversions
        betas = Pa2lbfsqf(spacecraft.beta)

    elif mission == "beta_study":
        input_units = "imperial"
        betas =  [100, 500, 1000, 5000]  # lbf/ft^2
        imperial_initial_conditions = [22500, 12.0, 0.0, 0.0] # [V_ini [ft], gamma_ini[deg], time_ini, range_ini]
        imperial_altitude = 250_000.0 # ft
        spacecraft = Spacecraft(beta_study=True, L_ref=1.0, R_nose = 1.0, imperial_units=True)
        initial_conditions = imperial_initial_conditions
    else:
        altitude = altitude
        initial_conditions = [V_0, gamma_0, 0.0, 0.0] 
        spacecraft = Spacecraft( W=W, A_ref=A_ref, L_ref=1.0, Cd=Cd, Cl=None, parachute=orion_parachute,
                                R_nose = R_nose, imperial_units=(input_units=="imperial"))
        
        betas = np.linspace(betas[0],betas[1], num=4) if custom_betastudy is not False else Pa2lbfsqf(spacecraft.beta)

    run_ballistic_simulation(betas=betas, 
                            initial_conditions=initial_conditions,
                            altitude=altitude, spacecraft=spacecraft,
                            input_units=input_units,
                                 plot_units=plot_units)



# Lifting Re-entry




Tots els gràfics es produeixen a partir de les solucions de velocitat, $\gamma$, temps, range, provinents de l'integració numérica de les següents equacions de reentrada sustentada:



$$ \frac{d V}{d t}=g\left[\frac{-Q}{\beta}+\sin (\gamma)\right] \\$$ 
$$ \frac{d \gamma}{d t}=\frac{-\frac{Q g}{\beta} \frac{L}{D}+\cos  (\gamma)\left[g-\frac{V^{2}}{\mathrm{R}_{\mathrm{e}+h}}\right]}{V} $$ 
$$ \frac{d h}{d t}=-V \sin (\gamma) $$

$$ \frac{d r}{d t}=\mathrm{R}_{\mathrm{e}} \frac{d \theta}{d t}=\frac{\mathrm{R}_{\mathrm{e}} V \cos (\gamma)}{\mathrm{R}_{\mathrm{e}}+h} $$




Seleccionant missio *custom* es poden triar tots els paràmetres de condicions inicials i d'especificacions del spacecraft. També es pot  triar un  rang de $\gamma$ per a fer un estudi similar al del paper, per a tal, cal  activar l'opció *custom_gammastudy*.

Escollint Space Shuttle obtenim la Figura 3 del paper.


In [ ]:
widgets.interact_manual.opts['manual_name'] = 'Run Simulation'
@widgets.interact_manual(gamma_0s= widgets.FloatRangeSlider(
    value=[0.5,3.0],
    min=0.05, max=3.0, step=0.2,
    disabled=False,
    description='Gamma_0s:',
    readout_format='.2f',
    ), custom_gammastudy=widgets.Checkbox(
    value=None,
    description='Custom gamma study',
    disabled=False,
    indent=True
))
def lifting_example(mission=["custom", "space_shuttle"], custom_gammastudy=False, input_units=["imperial", "metric"], plot_units=["imperial"], W=200000.0, 
                      L_ref=107.5, A_ref=2690.0, Cd=0.84, Cl=0.84,
                      R_nose=1.0, V_0=22500.0, gamma_0s=1.5, altitude=250000.0, time_lapse=1500):

    if mission == "space_shuttle":
        input_units = "imperial"
        W = 200_000 # lbf
        A_ref = 2690 # ft^2 m^2
        L_ref = 107.5 # ft
        R_nose = 1.0  # ft
        Cd = 0.84
        Cl = 0.84
        spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl, 
                                    parachute=False, imperial_units=True, beta_study=False )

        beta = Pa2lbfsqf(spacecraft.beta)

        altitude = 250_000
        V_0= 23_000.0
        gamma_0s= [0.1, 1.0, 2.5]
        time_lapse = 1500 # 1300 LSODA
        
    else:
        
        spacecraft = Spacecraft( W=W, A_ref=A_ref, L_ref=1.0, Cd=Cd, Cl=None, parachute=False,
                                R_nose = R_nose, imperial_units=(input_units=="imperial"))
        beta = Pa2lbfsqf(spacecraft.beta)
        gamma_0s = np.linspace(gamma_0s[0],gamma_0s[1], num=3) if custom_gammastudy is not False else gamma_0s
     
    
    run_lifting_simulation( beta=beta,
                     V_0=V_0, gamma_0s=gamma_0s,  
                     altitude=altitude, c_L=Cl, c_D=Cd, time_elapsed=time_lapse,
                     spacecraft=spacecraft,
                     input_units=input_units, solver="RK45")





---

# MaRV Example

Lifting entry is a key component to flight trajectories of a Maneuverable Re-entry Vehicle (MaRV) which flies a non-ballistic trajectory to help avoid intercept by defensive systems.

initial entry conditions of

Altitude = 250,000 ft 

Velocity = 22,500 ft/sec 

Flight Path Angle = 12 deg

- At 150,000 ft a constant L/D = 0.5 is applied to the MaRV trajectory for the remainder of the flight. 

- The MaRV continues along essentially a ballistic trajectory until about 70,000 ft, executes a pull-up (as lift becomes dominant over drag) to return to an altitude of about 140,000 ft, 
- Then flies basically a ballistic-like trajectory to earth impact. 


S'han pres  dades de longitud del vehicle, pes i $C_D$ derivats a partir de recerca de míssils balístics (B-61, B-661, RF-11),  que quadressin amb el  coeficient balístic de  1000lbf/sqft que apareix al paper.

In [ ]:
widgets.interact_manual.opts['manual_name'] = 'Run MaRV Simulation'
@widgets.interact_manual()
def marv_example():
    # B61 bomb specs

    W = 700 # lbf
    A_ref = 1.75 # ft^2 m^2
    L_ref = 12.5 # ft
    R_nose = 1.0  # ft
    Cd = 0.20
    Cl = 0.40
    spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl, 
                                parachute=False, imperial_units=True, beta_study=False )

    # beta = Pa2lbfsqf(spacecraft.beta)
    beta = 1000.0 #lbfsqf

    altitude = 250_000
    V_0= 22_500.0
    gamma_0s= [12]
    time_lapse = 400 # 1300 LSODA
    run_lifting_simulation( beta=beta,
                     V_0=V_0, gamma_0s=gamma_0s,  
                     altitude=altitude, c_L=Cl, c_D=Cd, time_elapsed=time_lapse,
                     spacecraft=spacecraft,
                     input_units="imperial", solver="RK45", marv=True)

